# Hard spheres in 2D

In [1]:
import numpy as np
import plotly.offline as py
from plotly.graph_objs import *

py.init_notebook_mode(connected=True)

%load_ext cython

In [2]:
DISK_RADIUS = 1


def collision(x, config, x_pos=-1):
    """Return True if the disk in position x collide with any other in config.
    optional parameter:
    - x_pos: the position of x in config (if present)
    """

    for i in range(len(config)):
        if i != x_pos and np.sqrt(np.sum((x-config[i])**2)) < 2*DISK_RADIUS:
            return True

    return False


def random_position(size):
    """Random position in a square (uniform sampling)."""
    return np.random.uniform(0, size, 2)


def configuration_direct_sampling(N, system_size):
    """Direct uniform extraction of N non-overlapping disks in a square."""
    config = []


    for i in range(N):
        x = random_position(system_size)
        y = [0, system_size, -system_size]
        x_pbc_clones = [ x + k for k in np.transpose([np.tile(y, len(y)), np.repeat(y, len(y))])]

        for j in x_pbc_clones:
            if collision(np.array(j) , np.array(config)):
                rejected_counter[0] += 1
                return configuration_direct_sampling(N, system_size)

        config += [x]

    return np.array(config)



def generate_square_lattice(system_size, num_disks):
    """Generates a square lattice with a given number of disks."""
    ss_disks = np.sqrt(num_disks)
    step = system_size / ss_disks
    x = [ step*(.5 + i) for i in range(int(ss_disks))]
    return np.transpose([np.tile(x, len(x)), np.repeat(x, len(x))])


def disk_shape(xc, yc, radius):
    x0 = xc - radius
    y0 = yc - radius
    x1 = xc + radius
    y1 = yc + radius

    return {
        'type': 'circle',
        'xref': 'x',
        'yref': 'y',
        'fillcolor': '#1F77B4',
        'line': dict(width=0),
        'x0': x0,
        'y0': y0,
        'x1': x1,
        'y1': y1,
    }

def plot_config(x, y, system_size):
    """Plot the configuration of the system."""
    trace = Scatter(
        x=x,
        y=y,
        mode='markers'
    )

    shapes = [{'type': 'square', 'x0': 0, 'y0': 0, 'x1': system_size, 'y1': system_size, 'xref':'x', 'yref':'y'}]
    
    for i in range(len(x)):
        shapes.append(disk_shape(x[i], y[i], DISK_RADIUS))
    
    lyt = Layout(
        yaxis=dict(scaleanchor="x", showgrid=False),
        xaxis=dict(showgrid=False),
        shapes=shapes
    )
    
    

    fig = dict(data=[trace], layout=lyt)

    py.iplot(fig)



def system_size(num_disks, density, disk_area):
    """Compute the system size to have that num_disks and density."""
    return np.sqrt(num_disks * disk_area / density)

def density(num_disks, syst_size):
    """Compute the density of disks (the fraction of area occupied by disks)."""
    return np.pi * num_disks / syst_size**2


def clone_collision(x, config, system_size):
    """Check if the disk is in the area in which is periodic clones can
       undergo collisions."""

    position = (x < 2*DISK_RADIUS) -1*(x > system_size - 2*DISK_RADIUS)

    if np.sum(np.abs(position)):
        clone = x + system_size*position
        if collision(clone, config):
            return True

    if np.sum(np.abs(position)) == 2:
        clone1 = x + system_size*np.array([0, position[1]])
        clone2 = x + system_size*np.array([position[0], 0])
        if collision(clone1, config) or collision(clone2, config):
            return True

    return False


def reposition_disk(x, system_size):
    """Reposition the disk inside the square if it's out."""
    out = np.array(-1*(x > system_size) + (x < 0))

    if np.sum(np.abs(out)):
        x += system_size * out

    return x


def move_disk(config, num_disks, step_size, system_size):
    """Try to move a random disk of the current config."""
    # move proposal
    disk = np.random.randint(num_disks)
    step = np.random.uniform(-step_size, step_size, 2)
    final_position = config[disk] + step

    # check if there are collision (considering also the PBCs)
    if (collision(final_position, config, disk)
        or clone_collision(final_position, config, system_size)):
        return config

    # reposition the disk if it is out
    final_position = reposition_disk(final_position, system_size)

    config[disk] = final_position

    return config


In [3]:
def _collision(x, config, x_pos=None):
    """Return True if the disk in position x collide with any other in config."""
    distances = np.sum(np.power(config - x, 2), axis=1)
    if x_pos is not None:
        distances[x_pos] = (3*DISK_RADIUS)**2
    
    return np.any(distances < (2*DISK_RADIUS)**2)


def _clone_collision(x, config, system_size):
    """Check if the disk is in the area in which is periodic clones can
       undergo collisions."""

    position = (x < 2*DISK_RADIUS) -1*(x > system_size - 2*DISK_RADIUS)

    if np.sum(np.abs(position)):
        clone = x + system_size*position
        if _collision(clone, config):
            return True

    if np.sum(np.abs(position)) == 2:
        clone1 = x + system_size*np.array([0, position[1]])
        clone2 = x + system_size*np.array([position[0], 0])
        if _collision(clone1, config) or _collision(clone2, config):
            return True

    return False

def _move_disk(config, num_disks, step_size, system_size):
    """Try to move a random disk of the current config."""
    # move proposal
    disk = np.random.randint(num_disks)
    step = np.random.uniform(-step_size, step_size, 2)
    final_position = config[disk] + step

    # check if there are collision (considering also the PBCs)
    if (_collision(final_position, config, disk)
        or _clone_collision(final_position, config, system_size)):
        return config

    # reposition the disk if it is out
    final_position = reposition_disk(final_position, system_size)

    config[disk] = final_position

    return config

In [4]:
%%cython

import numpy as np
cimport numpy as np
from libc.stdlib cimport rand, RAND_MAX

cdef double DISK_RADIUS = 1.

cdef bint c_collision(np.ndarray x, np.ndarray config, int x_pos=-1):
    """Return True if the disk in position x collide with any other in config."""
    cdef np.ndarray distances = np.sum(np.power(config - x, 2), axis=1)
    
    if x_pos >= 0:
        distances[x_pos] = (3*DISK_RADIUS)**2
        
    return np.any(distances < (2*DISK_RADIUS)**2)


cdef bint c_clone_collision(np.ndarray x, np.ndarray config, double system_size):
    """Check if the disk is in the area in which is periodic clones can
       undergo collisions."""

    cdef np.ndarray position = (x < 2*DISK_RADIUS) -1*(x > system_size - 2*DISK_RADIUS)
    cdef np.ndarray clone, clone1, clone2
    
    if np.sum(np.abs(position)):
        clone = x + system_size*position
        if c_collision(clone, config):
            return True

    if np.sum(np.abs(position)) == 2:
        clone1 = x + system_size*np.array([0, position[1]])
        clone2 = x + system_size*np.array([position[0], 0])
        if c_collision(clone1, config) or c_collision(clone2, config):
            return True

    return False

cdef np.ndarray c_reposition_disk(np.ndarray x, double system_size):
    """Reposition the disk inside the square if it's out."""
    cdef np.ndarray out = np.array(-1*(x > system_size) + (x < 0))

    if np.sum(np.abs(out)):
        x += system_size * out

    return x


cpdef np.ndarray c_move_disk(np.ndarray config, int num_disks, double step_size, double system_size):
    """Try to move a random disk of the current config."""
    # move proposal
    cdef int disk = rand() % num_disks
    cdef np.ndarray step = np.array([rand()/RAND_MAX, rand()/RAND_MAX])*2*step_size - step_size
    cdef np.ndarray final_position = config[disk] + step

    # check if there are collision (considering also the PBCs)
    if (c_collision(final_position, config, disk)
        or c_clone_collision(final_position, config, system_size)):
        return config

    # reposition the disk if it is out
    final_position = c_reposition_disk(final_position, system_size)

    config[disk] = final_position

    return config

In [5]:
import timeit

num_disks = 16**2
disk_area = np.pi * DISK_RADIUS**2
syst_size = system_size(num_disks, 0.5, disk_area)
config = generate_square_lattice(syst_size, num_disks)

def test_move_legacy():
    move_disk(config, num_disks, 0.5*DISK_RADIUS, syst_size)

def test_move():
    _move_disk(config, num_disks, 0.5*DISK_RADIUS, syst_size)

def test_cmove():
    c_move_disk(config, num_disks, 0.5*DISK_RADIUS, syst_size)

    
#config[0, 0] = 0

#_clone_collision(config[0], config, syst_size)
#plot_config(config[:, 0], config[:, 1], syst_size)

    
#print("LEG", timeit.timeit(test_move_legacy, number=1000))
print("UPD", timeit.timeit(test_move, number=1000))
print("C", timeit.timeit(test_cmove, number=1000))

UPD 0.14027489100044477
C 0.1161282700013544


We first try to implement a direct sampling strategy. We expect that reaching large density will be practically impossible, since the probability of obtaining two overlapping disk in the extracted configuration will be very high (the acceptance will tend to zero).

In [ ]:
rejected_counter = [0]
num_disks = 40
syst_size = 40
c = configuration_direct_sampling(num_disks, syst_size)

plot_config(c[:,0], c[:,1], syst_size)
print("For a density of disks of {0:4f} we get an acceptance ratio of 1/{1}". \
      format(density(num_disks, syst_size), rejected_counter[0] + 1))

As we can see even for small density the computational cost of obtaining a single configuration is very high. Also the presence of the periodic boundary condictions (PBCs) increases the computational cost since, for each step, we have to check if the new disk extracted collides with any other (due to the PBCs if a disk go beyond the square border can collide with the disks near the other side of the square). In this first approach we have controlled for each step if any of the particles duplicate (generated by the PBCs) collided with any other particle. In the following we'll do better, doing this check only when the disk is sufficiently near to the border (less than 2 times its radius).

## Markov chain Monte Carlo with square lattice initial condition


In [ ]:
NUM_STEPS = 100*10**6 + 1
num_disks = 16**2
disk_area = np.pi * DISK_RADIUS**2

for density in [0.5, 0.72]:
    syst_size = system_size(num_disks, density, disk_area)
    config = generate_square_lattice(syst_size, num_disks)

    for i in range(NUM_STEPS):
        config = c_move_disk(config, num_disks, 0.5*DISK_RADIUS, syst_size)
        
        if i % 10000 == 0:
            print("Step: {}".format(i))
        
        if i % (NUM_STEPS // 5) == 0:
            plot_config(config[:, 0], config[:, 1], syst_size)


#config after NUM_STEPS steps
# plot_config(config[:,0], config[:,1], syst_size)


Step: 0


Step: 10000
Step: 20000
Step: 30000
Step: 40000
Step: 50000
Step: 60000
Step: 70000
Step: 80000
Step: 90000
Step: 100000
Step: 110000
Step: 120000
Step: 130000
Step: 140000
Step: 150000
Step: 160000
Step: 170000
Step: 180000
Step: 190000
Step: 200000
Step: 210000
Step: 220000
Step: 230000
Step: 240000
Step: 250000
Step: 260000
Step: 270000
Step: 280000
Step: 290000
Step: 300000
Step: 310000
Step: 320000
Step: 330000
Step: 340000
Step: 350000
Step: 360000
Step: 370000
Step: 380000
Step: 390000
Step: 400000
Step: 410000
Step: 420000
Step: 430000
Step: 440000
Step: 450000
Step: 460000
Step: 470000
Step: 480000
Step: 490000
Step: 500000
Step: 510000
Step: 520000
Step: 530000
Step: 540000
Step: 550000
Step: 560000
Step: 570000
Step: 580000
Step: 590000
Step: 600000
Step: 610000
Step: 620000
Step: 630000
Step: 640000
Step: 650000
Step: 660000
Step: 670000
Step: 680000
Step: 690000
Step: 700000
Step: 710000
Step: 720000
Step: 730000
Step: 740000
Step: 750000
Step: 760000
Step: 770000
Step: 78

Step: 5940000
Step: 5950000
Step: 5960000
Step: 5970000
Step: 5980000
Step: 5990000
Step: 6000000
Step: 6010000
Step: 6020000
Step: 6030000
Step: 6040000
Step: 6050000
Step: 6060000
Step: 6070000
Step: 6080000
Step: 6090000
Step: 6100000
Step: 6110000
Step: 6120000
Step: 6130000
Step: 6140000
Step: 6150000
Step: 6160000
Step: 6170000
Step: 6180000
Step: 6190000
Step: 6200000
Step: 6210000
Step: 6220000
Step: 6230000
Step: 6240000
Step: 6250000
Step: 6260000
Step: 6270000
Step: 6280000
Step: 6290000
Step: 6300000
Step: 6310000
Step: 6320000
Step: 6330000
Step: 6340000
Step: 6350000
Step: 6360000
Step: 6370000
Step: 6380000
Step: 6390000
Step: 6400000
Step: 6410000
Step: 6420000
Step: 6430000
Step: 6440000
Step: 6450000
Step: 6460000
Step: 6470000
Step: 6480000
Step: 6490000
Step: 6500000
Step: 6510000
Step: 6520000
Step: 6530000
Step: 6540000
Step: 6550000
Step: 6560000
Step: 6570000
Step: 6580000
Step: 6590000
Step: 6600000
Step: 6610000
Step: 6620000
Step: 6630000
Step: 6640000
Step: 

Step: 11680000
Step: 11690000
Step: 11700000
Step: 11710000
Step: 11720000
Step: 11730000
Step: 11740000
Step: 11750000
Step: 11760000
Step: 11770000
Step: 11780000
Step: 11790000
Step: 11800000
Step: 11810000
Step: 11820000
Step: 11830000
Step: 11840000
Step: 11850000
Step: 11860000
Step: 11870000
Step: 11880000
Step: 11890000
Step: 11900000
Step: 11910000
Step: 11920000
Step: 11930000
Step: 11940000
Step: 11950000
Step: 11960000
Step: 11970000
Step: 11980000
Step: 11990000
Step: 12000000
Step: 12010000
Step: 12020000
Step: 12030000
Step: 12040000
Step: 12050000
Step: 12060000
Step: 12070000
Step: 12080000
Step: 12090000
Step: 12100000
Step: 12110000
Step: 12120000
Step: 12130000
Step: 12140000
Step: 12150000
Step: 12160000
Step: 12170000
Step: 12180000
Step: 12190000
Step: 12200000
Step: 12210000
Step: 12220000
Step: 12230000
Step: 12240000
Step: 12250000
Step: 12260000
Step: 12270000
Step: 12280000
Step: 12290000
Step: 12300000
Step: 12310000
Step: 12320000
Step: 12330000
Step: 1234

Step: 17150000
Step: 17160000
Step: 17170000
Step: 17180000
Step: 17190000
Step: 17200000
Step: 17210000
Step: 17220000
Step: 17230000
Step: 17240000
Step: 17250000
Step: 17260000
Step: 17270000
Step: 17280000
Step: 17290000
Step: 17300000
Step: 17310000
Step: 17320000
Step: 17330000
Step: 17340000
Step: 17350000
Step: 17360000
Step: 17370000
Step: 17380000
Step: 17390000
Step: 17400000
Step: 17410000
Step: 17420000
Step: 17430000
Step: 17440000
Step: 17450000
Step: 17460000
Step: 17470000
Step: 17480000
Step: 17490000
Step: 17500000
Step: 17510000
Step: 17520000
Step: 17530000
Step: 17540000
Step: 17550000
Step: 17560000
Step: 17570000
Step: 17580000
Step: 17590000
Step: 17600000
Step: 17610000
Step: 17620000
Step: 17630000
Step: 17640000
Step: 17650000
Step: 17660000
Step: 17670000
Step: 17680000
Step: 17690000
Step: 17700000
Step: 17710000
Step: 17720000
Step: 17730000
Step: 17740000
Step: 17750000
Step: 17760000
Step: 17770000
Step: 17780000
Step: 17790000
Step: 17800000
Step: 1781

Step: 20010000
Step: 20020000
Step: 20030000
Step: 20040000
Step: 20050000
Step: 20060000
Step: 20070000
Step: 20080000
Step: 20090000
Step: 20100000
Step: 20110000
Step: 20120000
Step: 20130000
Step: 20140000
Step: 20150000
Step: 20160000
Step: 20170000
Step: 20180000
Step: 20190000
Step: 20200000
Step: 20210000
Step: 20220000
Step: 20230000
Step: 20240000
Step: 20250000
Step: 20260000
Step: 20270000
Step: 20280000
Step: 20290000
Step: 20300000
Step: 20310000
Step: 20320000
Step: 20330000
Step: 20340000
Step: 20350000
Step: 20360000
Step: 20370000
Step: 20380000
Step: 20390000
Step: 20400000
Step: 20410000
Step: 20420000
Step: 20430000
Step: 20440000
Step: 20450000
Step: 20460000
Step: 20470000
Step: 20480000
Step: 20490000
Step: 20500000
Step: 20510000
Step: 20520000
Step: 20530000
Step: 20540000
Step: 20550000
Step: 20560000
Step: 20570000
Step: 20580000
Step: 20590000
Step: 20600000
Step: 20610000
Step: 20620000
Step: 20630000
Step: 20640000
Step: 20650000
Step: 20660000
Step: 2067

Step: 25480000
Step: 25490000
Step: 25500000
Step: 25510000
Step: 25520000
Step: 25530000
Step: 25540000
Step: 25550000
Step: 25560000
Step: 25570000
Step: 25580000
Step: 25590000
Step: 25600000
Step: 25610000
Step: 25620000
Step: 25630000
Step: 25640000
Step: 25650000
Step: 25660000
Step: 25670000
Step: 25680000
Step: 25690000
Step: 25700000
Step: 25710000
Step: 25720000
Step: 25730000
Step: 25740000
Step: 25750000
Step: 25760000
Step: 25770000
Step: 25780000
Step: 25790000
Step: 25800000
Step: 25810000
Step: 25820000
Step: 25830000
Step: 25840000
Step: 25850000
Step: 25860000
Step: 25870000
Step: 25880000
Step: 25890000
Step: 25900000
Step: 25910000
Step: 25920000
Step: 25930000
Step: 25940000
Step: 25950000
Step: 25960000
Step: 25970000
Step: 25980000
Step: 25990000
Step: 26000000
Step: 26010000
Step: 26020000
Step: 26030000
Step: 26040000
Step: 26050000
Step: 26060000
Step: 26070000
Step: 26080000
Step: 26090000
Step: 26100000
Step: 26110000
Step: 26120000
Step: 26130000
Step: 2614

Ater some trial we get an acceptance of about 0.5 for:
- a step size of 0.65*DISK_RADIUS for density=0.5
- a step size 0.11*DISK_RADIUS for density=0.72
for the a density of 0.5 nothing particular happens: the system gets disordered and then remains disordered, behaving like a liquid in some way.
When the density increases we can see that after long enough domains of ordered (aligned) particles arise in the system (that behaves like a solid).